In [198]:
import pandas as pd
from itertools import permutations
from numpy import cos, sin, arcsin, sqrt
from math import radians
import requests
import urllib
import time
import json

In [199]:
stations_2014 = pd.read_csv('data/2014/Stations_2014.csv')
stations_2015 = pd.read_csv('data/2015/Stations_2015.csv')
stations_2016 = pd.read_csv('data/2016/Stations_2016.csv')
stations_2017 = pd.read_csv('data/2017/Stations_2017.csv')
stations_2018 = pd.read_csv('data/2018/Stations_2018.csv')
stations_2019 = pd.read_csv('data/2019/Stations_2019.csv')
stations_2020 = pd.read_csv('data/2020/Stations_2020.csv')
stations_2021 = pd.read_csv('data/2021/Stations_2021.csv')

In [200]:
for station in [stations_2017, stations_2018, stations_2019]:
    print(len(station))

546
552
619


In [201]:
stations_2021 # no station code!

,pk,name,latitude,longitude
0,10,Métro Angrignon (Lamont / des Trinitaires),45.446910,-73.603630
1,13,Métro de l'Église (Ross / de l'Église),45.462726,-73.565959
2,17,Tolhurst / Fleury,45.544149,-73.667520
3,21,St-Christophe / Cherrier,45.520290,-73.568119
4,31,Lespérance / de Rouen,45.538839,-73.552779
...,...,...,...,...
824,1136,Terminus Le Carrefour (Terry-Fox / Le Carrefour),45.568420,-73.746873
825,1137,Métro Lionel-Groulx (St-Jacques / Atwater),45.483629,-73.579703
826,1138,Maguire / St-Dominique,45.525267,-73.595237
827,1139,CHU Ste-Justine (de la Côte Ste-Catherine / Hu...,45.503982,-73.622839


In [192]:
station_0 = stations_2019.iloc[:1]

In [193]:
station_0

,Code,name,latitude,longitude
0,10002,Métro Charlevoix (Centre / Charlevoix),45.478228,-73.569651


In [194]:
# density = 0
# for row in stations_2019.iterrows():
#     lon1 = station_0['longitude']
#     lat1 = station_0['latitude']

#     lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, row[1]['longitude'], row[1]['latitude']])
#     dlon = lon2 - lon1 
#     dlat = lat2 - lat1 
#     a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
#     c = 2 * arcsin(sqrt(a)) 
#     km = 6367 * c

#     if (km < 10):
#         print(km)
#         density += 1

# print(density)

In [195]:
# distance_frame = pd.DataFrame(columns=['base_station_code', 'target_station_code', 'distance'])
# density_frame = pd.DataFrame(columns=['station_code', 'density'])
# stations_2019_test = stations_2019.head(10)


# for index in permutations(stations_2019_test.index, 2):

#     base_station_code = stations_2019_test.at[index[0], 'Code']
#     lon1 = stations_2019_test.iloc[[index[0]]]['longitude']
#     lat1 = stations_2019_test.iloc[[index[0]]]['latitude']

#     target_station_code = stations_2019_test.at[index[1], 'Code']
#     lon2 = stations_2019_test.iloc[[index[1]]]['longitude']
#     lat2 = stations_2019_test.iloc[[index[1]]]['latitude']

#     lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
#     dlon = lon2 - lon1 
#     dlat = lat2 - lat1 
#     a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
#     c = 2 * arcsin(sqrt(a)) 
#     km = 6367 * c

#     distance_frame = distance_frame.append({
#         'base_station_code': base_station_code,
#         'target_station_code':target_station_code,
#         'distance': km
#     }, ignore_index=True)

#     distance_frame[['base_station_code', 'target_station_code']] = distance_frame[['base_station_code', 'target_station_code']].astype('int')

# unique_base_station_codes = list(distance_frame.base_station_code.unique())

# for base_station_code in unique_base_station_codes:

#     dict = {
#         'station_code': base_station_code,
#         'density': len(distance_frame[(distance_frame['base_station_code']==base_station_code) & (distance_frame['distance'] < 7)])
#     }

#     df2 = pd.DataFrame([dict])

#     density_frame = pd.concat([density_frame, df2])

# stations_2019_test = stations_2019_test.merge(density_frame, left_on='Code', right_on='station_code').drop(columns='station_code')

# stations_2019_test

In [196]:
def calculate_station_distances(station_df, threshhold):
    distance_frame = pd.DataFrame(columns=['base_station_code', 'target_station_code', 'distance'])
    i = 0
    distance_dict = {}

    for index in permutations(station_df.index, 2):

        base_station_code = station_df.at[index[0], 'Code']
        lon1 = station_df.iloc[[index[0]]]['longitude']
        lat1 = station_df.iloc[[index[0]]]['latitude']

        target_station_code = station_df.at[index[1], 'Code']
        lon2 = station_df.iloc[[index[1]]]['longitude']
        lat2 = station_df.iloc[[index[1]]]['latitude']

        # calculate distance
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
        dlon = lon2 - lon1 
        dlat = lat2 - lat1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * arcsin(sqrt(a)) 
        km = 6367 * c

        if km > threshhold:
            continue
        else:
            distance_dict[i] = {
            'base_station_code': base_station_code,
            'target_station_code':target_station_code,
            'distance': km
        }
            i += 1
    
    distance_frame = pd.DataFrame.from_dict(distance_dict, "index")

    distance_frame[['base_station_code', 'target_station_code']] = distance_frame[['base_station_code', 'target_station_code']].astype('int')

    distance_frame = distance_frame[distance_frame['distance'] < threshhold]


    
    return distance_frame

In [197]:
distance_frame = calculate_station_distances(stations_2021, 1)

KeyError: 'Code'

In [ ]:
distance_frame

,base_station_code,target_station_code,distance
0,6001,6003,0.823640
1,6001,6004,0.212324
2,6001,6007,0.459403
3,6001,6008,0.476322
4,6001,6011,0.538046
...,...,...,...
15613,8036,8053,0.890346
15614,8069,7131,0.579556
15615,8022,7128,0.724348
15616,8022,7133,0.846200


In [ ]:
def get_processed_stations_density_frame(distance_frame):
    unique_base_station_codes = list(distance_frame.base_station_code.unique())

    i = 0
    density_dict = {}

    for base_station_code in unique_base_station_codes:

        density_dict[i] = {
            'station_code': base_station_code,
            'density': len(distance_frame[(distance_frame['base_station_code']==base_station_code)])
        }
        i+=1


    density_frame = pd.DataFrame.from_dict(density_dict, "index")

    stations_processed = stations_2019.merge(density_frame, left_on='Code', right_on='station_code', how='inner').drop(columns='station_code')

    return stations_processed

In [ ]:
stations_processed = get_processed_stations_density_frame(distance_frame)

In [ ]:
# stations_2019_processed = stations_2019.merge(density_frame, left_on='Code', right_on='station_code', how='inner').drop(columns='station_code')

In [ ]:
stations_processed

,Code,name,latitude,longitude,density
0,10002,Métro Charlevoix (Centre / Charlevoix),45.478228,-73.569651,18
1,4000,Jeanne-d'Arc / Ontario,45.549598,-73.541874,13
2,4001,Graham / Brookfield,45.520075,-73.629776,7
3,4002,Graham / Wicksteed,45.516937,-73.640483,3
4,5002,St-Charles / Montarville,45.533682,-73.515261,3
...,...,...,...,...,...
604,7147,Métro de Castelnau (de Castelnau / Clark),45.534739,-73.620769,28
605,7148,Papineau / Émile-Journault,45.559884,-73.633568,2
606,7149,16e avenue / Jean-Talon,45.562440,-73.595333,4
607,7152,Parc St-Claude (7e avenue / 8e rue),45.562668,-73.692362,2


In [ ]:
stations_processed.describe()

,Code,latitude,longitude,density
count,609.000000,609.000000,609.000000,609.000000
mean,6478.129721,45.521126,-73.585832,25.224959
std,470.359942,0.032749,0.034757,17.218993
min,4000.000000,45.431471,-73.711481,1.000000
25%,6159.000000,45.501367,-73.605560,8.000000
50%,6345.000000,45.523194,-73.577856,24.000000
75%,6925.000000,45.540060,-73.564315,41.000000
max,10002.000000,45.651406,-73.490113,58.000000


In [ ]:
url = r'https://api.open-elevation.com/api/v1/lookup?'

def elevation_function(df, lat_column, lon_column):
    elevations = []
    for lat, lon in zip(df[lat_column], df[lon_column]):
        
        coords = str(lat) + "," + str(lon)

        params = {
            'locations': coords,
        }
        
        result = requests.get((url + urllib.parse.urlencode(params)))
        print(result)
        elevations.append(result.json()['results'][0]['elevation'])
        time.sleep(0.2)

    df['elevation_meters'] = elevations

In [ ]:
elevation_function(stations_processed, 'latitude', 'longitude')

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [ ]:
stations_processed

,Code,name,latitude,longitude,density,elevation_meters
0,10002,Métro Charlevoix (Centre / Charlevoix),45.478228,-73.569651,18,16
1,4000,Jeanne-d'Arc / Ontario,45.549598,-73.541874,13,21
2,4001,Graham / Brookfield,45.520075,-73.629776,7,56
3,4002,Graham / Wicksteed,45.516937,-73.640483,3,53
4,5002,St-Charles / Montarville,45.533682,-73.515261,3,16
...,...,...,...,...,...,...
604,7147,Métro de Castelnau (de Castelnau / Clark),45.534739,-73.620769,28,59
605,7148,Papineau / Émile-Journault,45.559884,-73.633568,2,40
606,7149,16e avenue / Jean-Talon,45.562440,-73.595333,4,50
607,7152,Parc St-Claude (7e avenue / 8e rue),45.562668,-73.692362,2,27


In [ ]:
stations_processed.to_csv('data/stations_preprocessed/Stations_2021.csv')

In [ ]:
processed_2019 = pd.read_csv('data/stations_preprocessed/Stations_2019.csv')
processed_2020 = pd.read_csv('data/stations_preprocessed/Stations_2020.csv')

In [204]:
processed_2019.describe()

,Unnamed: 0,Code,latitude,longitude,density,elevation_meters
count,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000
mean,304.000000,6476.981938,45.520971,-73.585708,25.126437,44.390805
std,175.947435,469.636027,0.032641,0.034696,17.328733,20.830653
min,0.000000,4000.000000,45.431471,-73.711481,1.000000,2.000000
25%,152.000000,6159.000000,45.501041,-73.604973,8.000000,26.000000
50%,304.000000,6345.000000,45.523170,-73.577827,24.000000,48.000000
75%,456.000000,6924.000000,45.539984,-73.564315,40.000000,59.000000
max,608.000000,10002.000000,45.651406,-73.490113,60.000000,124.000000


In [205]:
processed_2020.describe()

,Unnamed: 0,Code,latitude,longitude,density,elevation_meters
count,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000
mean,304.000000,6478.129721,45.521126,-73.585832,25.224959,44.384236
std,175.947435,470.359942,0.032749,0.034757,17.218993,20.834881
min,0.000000,4000.000000,45.431471,-73.711481,1.000000,2.000000
25%,152.000000,6159.000000,45.501367,-73.605560,8.000000,26.000000
50%,304.000000,6345.000000,45.523194,-73.577856,24.000000,48.000000
75%,456.000000,6925.000000,45.540060,-73.564315,41.000000,59.000000
max,608.000000,10002.000000,45.651406,-73.490113,58.000000,124.000000
